# FP-growth
- FP-growth算法
    - 优点：快于Apriori
    - 缺点：实现比较困难，在某些数据集上性能下降
    - 适用数据类型：标称型
- FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中
    - FP树存储项集出现的频率，每个项集以路径的方式存储在树中
    - 相似元素的集合共享树的一部分
    - 一个FP树对应一个最小支持度，小于最小支持度
    - 构建FP树需要对数据做两遍扫描，第一次统计出现的频率，第二次只考虑那些频繁元素

## 构建FP树

In [1]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self,numOccur):
        self.count += numOccur
    
    def disp(self,ind=1):
        print('  '*ind,self.name,'  ',self.count)
        for child in self.children.values():
            child.disp(ind+1)       

In [2]:
rootNode = treeNode('pramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.children['phoenix'] = treeNode('phoenix',3,None)

rootNode.disp()

   pramid    9
     eye    13
     phoenix    3


- 除了上述的数据结构之外，还需要一个头指针指向给定类型的第一个实例。利用头指针，可以快速访问FP树中的一个给定类型的所有元素，此外头指针还可以用来保存FP树中每类元素的数目
- 第一次遍历数据集挥霍的每个元素项出现的频率，随后去掉不满足最小支持度的元素项，再进一步构建FP树
- 构建时，读入每个项集并添加到一条已经存在的路径中。为了方便需要在集合添加到树之前对每个元素排序，排序基于绝对出现频率进行

In [3]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [4]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [5]:
# 将新的元素挂接到头指针表
def updateHeader(nodeToTest,tragetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = tragetNode

def updateTree(items,inTree,headerTable,count):
    # 处理频率最高元素
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0],count,inTree)
    
        # 链接头指针列表
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
        
    # 对剩下的元素迭代更新
    if len(items)>1:
        updateTree(items[1::],inTree.children[items[0]],headerTable,count)

In [6]:
def createTree(dataSet,minSup=1):
    # 初始化元素频率集合
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item,0) + dataSet[trans]
    # 筛选不满足条件的
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0: 
        return None, None
    
    # 修正头指针表
    for k in headerTable:
        headerTable[k] = [headerTable[k],None]


    retTree = treeNode('Null Set',1,None)
    for tranSet,count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:#这里改成大于minSup也等价
            orderedItems = [v[0] for v in sorted(localD.items(),key=lambda  p:p[1],reverse=True)]
            updateTree(orderedItems,retTree,headerTable,count)            
    return retTree,headerTable


In [7]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [8]:
myFPtree,myHeaderTab = createTree(initSet,3)

In [9]:
myFPtree.disp()

   Null Set    1
     z    5
       r    1
       x    3
         t    3
           y    3
             s    2
             r    1
     x    1
       s    1
         r    1


# 从FP树中挖掘频繁项集
1. 从FP树中获取条件模式基
2. 利用条件模式基构建一个条件FP树
3. 迭代直到树只有一个元素

## 抽取条件模式基
- 对于头指针表仲每个频繁单个元素项，获得对应的**条件模式基**（conditional pattern base）
- 条件模式基是以查找元素为结尾的路径集合，每一条路径其实都是一条**前缀路径**（prefix path）
- 可以通过穷举搜索得到前缀路径，也可以使用更有效的方法加速搜索过程（利用头指针包含想同类型元素链表的启事指针，一旦达到了某一个元素项，就可以上溯到这棵树的根节点为止）


In [10]:
# 发现给顶元素项结尾的所有路径函数

# 迭代向上回溯整个树
def ascendTree(leafNode,prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)
        
def findPrefixPath(basePat,treeNode):
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode,prefixPath)
        if len(prefixPath)>1:
            condPats[frozenset(prefixPath[1::])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [11]:
findPrefixPath('x',myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [12]:
findPrefixPath('z',myHeaderTab['z'][1])

{}

In [13]:
findPrefixPath('r',myHeaderTab['r'][1])

{frozenset({'z'}): 1,
 frozenset({'s', 'x'}): 1,
 frozenset({'t', 'x', 'y', 'z'}): 1}

## 创建条件FP树
- 对于每一个频繁项，创建一个条件FP树
- 使用条件模式基作为输入，并构建对应的树

In [14]:
def mineTree(inTree,headerTable,minSup,preFix,freqitemList):
    bigL = [v[0] for v in sorted(headerTable.items(),key=lambda p:p[0])]
    
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqitemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat,headerTable[basePat][1])
        myCondTree,myHead = createTree(condPattBases,minSup)
        
        if myHead != None:
            print('conditional tree for :',newFreqSet)
            myCondTree.disp()
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqitemList)
        

In [15]:
freqItems = []

In [16]:
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

conditional tree for : {'s'}
   Null Set    1
     x    3
conditional tree for : {'t'}
   Null Set    1
     z    3
       x    3
conditional tree for : {'t', 'x'}
   Null Set    1
     z    3
conditional tree for : {'x'}
   Null Set    1
     z    3
conditional tree for : {'y'}
   Null Set    1
     t    3
       z    3
         x    3
conditional tree for : {'y', 'x'}
   Null Set    1
     t    3
       z    3
conditional tree for : {'z', 'y', 'x'}
   Null Set    1
     t    3
conditional tree for : {'z', 'y'}
   Null Set    1
     t    3


In [17]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'t', 'y'},
 {'x', 'y'},
 {'t', 'x', 'y'},
 {'x', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'y', 'z'},
 {'t', 'y', 'z'},
 {'z'}]

# e.g. 从新闻网站点击流中挖掘

In [18]:
parsedDat = [line.split() for line in open('../data/FP_growth/kosarak.dat').readlines()]

In [19]:
initSet = createInitSet(parsedDat)

In [20]:
myFPtree,myHeaderTab = createTree(initSet,100000)

In [21]:
myFreqList = []
mineTree(myFPtree,myHeaderTab,100000,set([]),myFreqList)

conditional tree for : {'1'}
   Null Set    1
     6    107404
conditional tree for : {'11'}
   Null Set    1
     6    261773
conditional tree for : {'3'}
   Null Set    1
     6    186289
       11    117401
     11    9718
conditional tree for : {'3', '11'}
   Null Set    1
     6    117401


In [22]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]